In [6]:
!pip install tensorflow

In [7]:
!pip install tflearn

     ---------------------------------------- 0.0/107.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/107.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/107.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/107.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/107.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/107.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/107.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/107.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/107.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/107.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/107.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/107.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/107.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/1

In [8]:
!pip install tqdm

In [1]:
import cv2
import numpy as np
import os         
from random import shuffle 
from tqdm import tqdm      
import tensorflow as tf
import matplotlib.pyplot as plt

%matplotlib inline

TRAIN_DIR="C:/Users/User/python練習/train/train"
TEST_DIR="C:/Users/User/python練習/test/test"
IMG_SIZE=50
LR=1e-3
MODEL_NAME='dogs-vs-cats-convnet'

In [2]:
def label_img(img):
    word_label=img.split('.')[0]
    if word_label=='cat':return[1,0]
    elif word_label=='dog':return[0,1]

In [3]:
def create_train_data():
    training_data=[]
    for img in tqdm(os.listdir(TRAIN_DIR)):
        labol=label_img(img)
        path=os.path.join(TRAIN_DIR,img)
        img=cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE),(IMG_SIZE, IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy',training_data)
    return training_data

In [4]:
def create_test_data():
    testing_data=[]
    for img in tqdm(os.listdir(TEST_DIR)):
        path=os.path.join(TEST_DIR,img)
        img_num=img.split('.')[0]
        img_data=cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE),(IMG_SIZE, IMG_SIZE))
        testing_data.append([np.array(img),img_num])
    shuffle(testing_data)
    np.save('train_data.npy',training_data)
    return training_data

In [5]:
train_data=create_train_data()
test_data=create_test_data()

  0%|          | 0/11941 [00:00<?, ?it/s]


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [ ]:
import tflearn
from tflearn.layers.conv import conv_2d,max_pooi_2d
from tflearn.layers.core import input_data,dropout,fully_connected
from tflearn.layers.estimator import regression

In [7]:
convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')
convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)
convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')
model = tflearn.DNN(convnet, tensorboard_dir='log', tensorboard_verbose=0)

NameError: name 'input_data' is not defined

In [ ]:
if os.path.exists('{}.meta'.format(MODEL _NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

In [ ]:
train=train_data[:-8000]
test=train_data[-8000:]

In [ ]:
X=np.array(i[0]for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y=[i[1]for i in train]
test_x=np.array(i[0]for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y=[i[1]for i in train]

In [ ]:
model.fit({'input':X},{'targets':Y},n_epoch=5,
          validation_set=({'input':test_x),{'targets':test_y}),
          snapshot_step=500,show_metric=True,run_id=MODEL_NAME)

In [ ]:
model.save(MODEL_NAME)

In [ ]:
d=test_data[0]
img_data, img_num = d
data = img_data.reshape(IMG_SIZE, IMG_SIZE, 1)
prediction = model.predict([data])[0]
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111)
ax.imshow(img_data, cmap="gray")
print（'Cat Possibility：{：5.2f｝%\nDog Possioiity：{:5.2f}%'.format(prediction[0]*100,prediction[1]*100))

In [ ]:
fig=plt.figure(figsize=(16, 12))
for num, data in enumerate(test_data[:16]):
    img_num = data[1]
    img_data = data[0]
    y = fig.add_subplot(4, 4, num+1)
    orig = img_data
    data = img_data.reshape(IMG_SIZE, IMG_SIZE, 1)
    model_out = model.predict([data])[0]
    if np.argmax(model_out) == 1: 
        str_label='Dog'
    else:
        str_label='Cat'
        
    y.imshow(orig, cmap='gray')
    plt.title(str_label)
    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)
plt.show()